# Duplication

Пример отчета по пересечению аудитории проектов Web-Index.

## Описание задания

Задача посчитать пересечение аудитории проектов:
- Vk.com
- Odnoklassniki.ru

Общие параметры:
- Период: Февраль 2020
- География: Россия 100+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop,Web Mobile,App Online,App Offline).

Расчитываем ститистики:

- Общие для выборки:
    - Unweighted Reach

- Для пересечения проектов:
    - Reach
    - ReachColumn%
    - ReachRow%


Для расчета данного отчета потребуется сформировать два задания для Responsum:
 
    1. для расчета общих статистик
    2. для расчета пересечения


In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging
import seaborn as sns
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

Loading BokehJS ...

## Общие параметры для заданий

Для начала зададим общие параметры

In [2]:
# Задаем параметры
# выбираем тип установки mobile.
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)


# задаем период расчета
date_from = '2020-02-01'
date_to = '2020-02-29'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, нас интересует города с населением 100тыс. и больше
population = rcats.get_population('Russia100+')


# задаем Возврастные группы, нас интересует 12+
ages = rcats.get_age_groups('12+')


# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:14:44
Тип установки: mobile
Начало периода: 2020-02-01
Конец периода: 2020-02-29
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3, 4]
Возрастные группы: [1, 2, 3, 4, 5, 6, 7]
Объектов в media-каталоге: 4716


### Получим ID проектов
Для построения отчета нам необходимо получить идентификаторы сайтов __Vk.com__ и __Odnoklassniki.ru__, для этого воспользуемся блокнотом, в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Vk.com  site_id = 16571
- Odnoklassniki.ru site_id = 12808

## Задания
Перейдем к формированию заданий


### Задание №1 Расчет пересечения аудитории проектов

In [3]:
# Задаем название проекта для отображения в DataFrame
project_name = 'dup'

# Задаем медиа фильтр и duplication фильтр, в нашем случае это ID проекта VK.com и Odnoklassniki.ru
media_filter = "site = 16571"
dup_media_filter = "site = 12808"

# Фильтр по демографии отсутствует
demo_filter = None

# Задаем список статистик для расчета
statistics=["Reach"]

# Указываем порядок группировки
structure =  {
    "usetype": False,
    "media": ["site"],
    "duplication": ["site"]
  }

# Формируем из заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_duplication_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, 
                                         population_filter=population, ages_filter=ages, media_filter=media_filter, dup_media_filter=dup_media_filter, 
                                         demo_filter=demo_filter, statistics=statistics, structure=structure)

# Отправляем задание на расчет и ждем выполнения
task_dup = rtask.wait_task(rtask.send_duplication_task(task_json))

#Получаем результат
df_dup = rtask.result2table(rtask.get_result(task_dup), project_name=project_name)
# df_uni
df_dup

Задача <967c4e8b-0e95-4016-87a4-50d2e6324c46> поступила в обработку
Расчет задачи [= = = ] время расчета: 0:00:10.378705


,prj_name,media_site,duplication_site,stat_reach
0,dup,Vk.com,Odnoklassniki.ru,41590.329466


Посчитаем дополнительные отчеты, необходимые для расчета процентов


### Задание №2 Расчет Total для проектов

In [4]:
# Задаем название проекта для отображения в DataFrame
project_name = 'total_project'

# Задаем фильтр по медии
media_filter = "site = 16571 OR site = 12808"

# Фильтр по демографии отсутствует
demo_filter = None

# Задаем список статистик для расчета
statistics=["Reach"]

# Указываем порядок разбивки по сайтам
structure =  {
    "usetype": False,
    "media": ["site"]
  }
# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# Получаем результат
df_total_prj = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total_prj

Задача <864b648c-b20c-45ac-a30e-82ab58196d0e> поступила в обработку
Расчет задачи [= = = = = ] время расчета: 0:00:16.717272


,prj_name,media_site,stat_reach
0,total_project,Vk.com,69807.996147
1,total_project,Odnoklassniki.ru,52291.987049


### Задание №3 Расчет TotalInternet для заданного периода


In [5]:
# Задаем название проекта для отображения в DataFrame
project_name = 'total'

# Фильтр по медиа отсутствует
media_filter = None

# Фильтр по демографии отсутствует
demo_filter = None

# Задаем список статистик для расчета
statistics=["Reach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <3e61c147-322e-48b7-bab0-8d3cc998019d> поступила в обработку
Расчет задачи [= = = = ] время расчета: 0:00:13.653158


,prj_name,stat_reach
0,total,94518.316163


### Формирем итоговую таблицу

Объединим полученные результаты и посчитаем доли для каждой из цифр

In [6]:
df_duplication = rtask.calc_duplication_row_col(df_dup, df_total_prj, df_total)
df_duplication

12:15:45 INFO:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
12:15:45 INFO:NumExpr defaulting to 8 threads.


,media_site,duplication_site,duplication_stat_reach,media_stat_reach,stat_prc_reach
0,Vk.com,Odnoklassniki.ru,41590.329466,69807.996147,59.578174
1,Odnoklassniki.ru,Vk.com,41590.329466,52291.987049,79.534804
2,Vk.com,total,69807.996147,94518.316163,73.856580
3,Odnoklassniki.ru,total,52291.987049,94518.316163,55.324713


#### Округлим полученные занчения до второго знака.
Воспользуемся стандартным методом библиотеки pandas

In [7]:
df_duplication[['rnd_reach', 'rnd_prc_reach']] = df_duplication[['media_stat_reach', 'stat_prc_reach']].round(2)
df_duplication

,media_site,duplication_site,duplication_stat_reach,media_stat_reach,stat_prc_reach,rnd_reach,rnd_prc_reach
0,Vk.com,Odnoklassniki.ru,41590.329466,69807.996147,59.578174,69808.00,59.58
1,Odnoklassniki.ru,Vk.com,41590.329466,52291.987049,79.534804,52291.99,79.53
2,Vk.com,total,69807.996147,94518.316163,73.856580,94518.32,73.86
3,Odnoklassniki.ru,total,52291.987049,94518.316163,55.324713,94518.32,55.32


## Экспорт в Excel

In [8]:
writer = pd.ExcelWriter(rtask.get_excel_filename('duplication-VK-OK'))
df_info = rtask.get_report_info()
df_duplication.to_excel(writer, 'Report')
df_info.to_excel(writer, 'Info', index=False)
writer.save()